<h1>遺伝子(CDS)上の非同義置換アミノ酸(Nonsynonymous amono acid)探索</h1>

連絡先：t.n100409@gmail.com

<p>同じコードを複数回実行するとエラーが出る場合や結果が異なってしまう場合があります。</p>
<p>上部のランタイムからランタイムを再起動し、最初から実行し直すと解決できます。</p>

自分で変更する必要のある部分は<b><font color=red>$$$</font></b>で挟んである

<h2>必要データ</h2>

<p>mRNAのcsvファイル</p>
<p>全塩基配列のテキストファイル</p>
<p>ユーロフィンデータ</p>

<h2>生産物</h2>



*   ４パターンの塩基配列
*   ４パターンのアミノ酸配列
*   変異前後の塩基、コドン、アミノ酸一覧



<h2>1.初期設定</h2>




<h3>1.1 ライブラリのインポート</h3>

In [ ]:
import pandas as pd
import numpy as np
import itertools
import openpyxl
import glob
import math
import os
import copy
from google.colab import drive

<h3>1.2 ドライブをマウント</h3>

・ドライブをマウントし、colabでドライブ内のファイルにアクセスできるようにする。

実行後、URLをクリックしてログインする。
'Mounted at /content/drive'と表示されれば表示されればOK

In [ ]:
drive.mount('/content/drive')

<h3>1.3 作業用ディレクトリの作成</h3>

・作業用ディレクトリをdrive/Mydrive直下に作成。

In [ ]:
os.makedirs('drive/My Drive/syn_detect', exist_ok=True)
field_dir = 'drive/My Drive/syn_detect'
data_dir = 'drive/My Drive/syn_detect/original_data'
work_dir = 'drive/My Drive/syn_detect/work'
os.makedirs(data_dir, exist_ok=True)
os.makedirs(work_dir, exist_ok=True)

<b><font color=red>$$$</font></b>

作成後、'drive/My Drive/syn_detect/original_data'の中に必要ファイルを入れる。

<b><font color=red>$$$</font></b>

・必要ファイル一覧

<p>mRNAのcsvファイル</p>
<p>全塩基配列のテキストファイル</p>
<p>ユーロフィンデータ</p>

<h3>1.4 データを読み込む</h3>

各データの仕様説明


1.   targetname:NCBIのGenbankのgeneID
2.   mRNAfilename:スプライシングの開始点と終了点情報
3.   sequencename:和金のゲノム配列(Reference)
4.   snpindel:ユーロフィンジェノミクスのデータ
※1,2,4は同じ染色体のものとする。



<b><font color=red>$$$</font></b>

In [ ]:
target_name = 'LOC113049214'
mRNAfilename = 'mRNAstartend.csv'
sequencename = 'Wakinsequence.txt'
snpindel = 'Sample_all_filtered_DP10GQ10AD_NC_039272.1.csv'

保存するフォルダ作成用に指定

In [ ]:
chromosome_number = 'ch30'

<b><font color=red>$$$</font></b>

<h2>2.mRNAファイルに関する処理</h2>

<h3>2.1 mRNAfileの読み込み</h3>

In [ ]:
df_mRNA = pd.read_csv(f'{data_dir}/{mRNAfilename}')

<font color='blue'>読み込んだmRNAfileを表示</font>

In [ ]:
df_mRNA

<h3>2.2 region=CDS&geneIDを設定値で抽出</h3>

In [ ]:
id_firstidxname = df_mRNA.columns[6] #excelファイルの7列1行目を指定している。
print(id_firstidxname)

'ID=数字'が表示されているか確認。ID列の一番上のセル。

<h3>2.3 抽出したテーブル作成</h3>

CDSかつgeneIDが設定値のものを抽出し、データ作成。

In [ ]:
df_cds = df_mRNA[(df_mRNA['region'] == 'CDS') & (df_mRNA['genome=chromosome'] == f'gene={target_name}')]

抽出項目を変更したい場合、上のコードを変更する。 例:'CDS' → 'exon'

<font color='blue'>CDS,IDが指定と一致するものを表示</font>

In [ ]:
df_cds

<h3>2.4 抽出データの処理</h3>

・抽出した中のIDをリスト化

In [ ]:
id_list = df_cds[id_firstidxname].unique()

<font color='blue'>一致したIDリストを表示</font>

In [ ]:
print(id_list)

・+or-を特定

<font color='blue'>+か-かを表示</font>

In [ ]:
for i in id_list:
  plmin = df_cds[df_cds[id_firstidxname] == i]['+']
  print(i)
  print(plmin)

<h3>2.5 特定のIDを見る</h3>

・IDリストのn番目だけ抽出し、startのPOS番号を昇順でソート

In [ ]:
print(id_list)

・下のコード(id_list_n)の数字を変更することで見るIDを変える。
0番目～n番目で指定する。




<b><font color=red>$$$</font></b>

In [ ]:
id_list_n = 0

<b><font color=red>$$$</font></b>

・指定したIDのテーブルを作成。

In [ ]:
df_group = df_cds[df_cds[id_firstidxname] == id_list[id_list_n]].sort_values('1')

<font color='blue'>指定したIDのテーブルを表示。</font>

In [ ]:
df_group

<h2>3.sequenceに関する処理</h2>

<h3>3.1 sequenceのテキストファイルの読み込み</h3>

sequenceテキストファイルの上から何行削除するかをdel_countで指定。
削除しない場合はは0にする。


<b><font color=red>$$$</font></b>

In [ ]:
del_count = 1

<b><font color=red>$$$</font></b>

sequence作成

In [ ]:
sequence = ''
with open(f'{data_dir}/{sequencename}') as f:
    for line in f:
        if del_count > 0:
          del_count -= 1
        else:
          line = line.rstrip()
          sequence += line

<font color='blue'>sequenceを表示</font>

文字数が多く、動作が遅くなるため非表示中。

表示する場合は文頭の#を削除して実行する。

In [ ]:
#sequence

<h3>3.2 途切れたシーケンスを結合</h3>

2で指定した同一IDの途切れたシーケンス同士を結合する。

In [ ]:
merge_sequence = ''
sp_sequence = []
startend_list = []
for idx_num in range(len(df_group)):
  startend = [df_group.loc[df_group.index[idx_num], '1'], df_group.loc[df_group.index[idx_num], str(len(sequence))]]
  seq = sequence[startend[0]-1: startend[1]]
  merge_sequence += seq
  sp_sequence.append(seq)
  startend_list.append(startend)

<p>merge_sequence : 全部をつなげたもの</p>
<p>sp_sequence : 1区画ごとにつなげたもの</p>
<p>startend_list : startとendのリスト</p>

<font color='blue'>各sequenceを表示</font>

In [ ]:
merge_sequence

In [ ]:
sp_sequence

In [ ]:
startend_list

<h2>4.クロデメキン染色体n番ユーロフィンデータの読み込み</h2>

読み込みに少し時間かかります。csvで読み込むように書き換えると速いです。

In [ ]:
df_sample = pd.read_csv(f'{data_dir}/{snpindel}')

<font color='blue'>クロデメキン染色体n番ユーロフィンデータの表示</font>

In [ ]:
df_sample

・扱いやすいように要素ごとに取得

In [ ]:
pos = pd.Series(df_sample['POS'])
ref = pd.Series(df_sample['REF'])
alt = pd.Series(df_sample['ALT'])
SNPorINDEL = pd.Series(df_sample['SNPorINDEL'])
kd_m_gt = pd.Series(df_sample['KD-M_GT'])
f1_f_gt = pd.Series(df_sample['F1-F_GT'])
num_gt = pd.Series(df_sample['Num_GT'])
kd_m_af_genotype_0 = pd.Series(df_sample['KD-M_AF_Genotype_0'])
kd_m_af_genotype_1 = pd.Series(df_sample['KD-M_AF_Genotype_1'])
f1_f_af_genotype_0 = pd.Series(df_sample['F1-F_AD_Genotype_0'])
f1_f_af_genotype_1 = pd.Series(df_sample['F1-F_AD_Genotype_1'])

<h3>4.1 start,endの変動チェック</h3>

<h4>4.1.1 start,endの前後2塩基塩基のSNPの有無をチェック</h4>

start,endの前後前後2塩基にSNPが存在し、終止コドンになる場合を想定して検査。

・startendの前後2塩基に変異がないか確認

In [ ]:
#前後２塩基にSNPがないことを確認(０なら該当なし)
a = 0
for i in range(len(startend_list)):
  a += len(df_sample[df_sample['POS'] == startend_list[i][0]-1])
  a += len(df_sample[df_sample['POS'] == startend_list[i][0]-2])
  a += len(df_sample[df_sample['POS'] == startend_list[i][1]+1])
  a += len(df_sample[df_sample['POS'] == startend_list[i][1]+2])
print(a)
del a

<h4>4.1.2 start,endに干渉し得るINDELを抽出</h4>

start前,end後のエキソン領域外にあるINDELのうち、start,endに干渉し得るものを抽出する。

・startととendの１個前、１個後のPOSを取得。さらにINDELのものだけ抽出

In [ ]:
indel_st_list = []
indel_ed_list = []
for i in range(len(startend_list)):
  for j in range(len(pos)):
    if startend_list[i][0] <= pos[j]:
      pos_id = pos[j-1]
      break
  if SNPorINDEL[df_sample[df_sample['POS'] == pos_id].index[0]] == 'SNP':
    pass
  else:
    indel_st_list.append([i,pos_id])

for i in range(len(startend_list)):
  for j in range(len(pos)):
    if startend_list[i][1] <= pos[j]:
      pos_id = pos[j]
      break
  if SNPorINDEL[df_sample[df_sample['POS'] == pos_id].index[0]] == 'SNP':
    pass
  else:
    indel_ed_list.append([i, pos_id])

・start,endと１個前後のINDELの塩基数の差を算出。ref,altと塩基数を比較。ref,altの方が小さければ排除。ref,altより大きいものを抽出

In [ ]:
indel_st_d = []
indel_ed_d = []
alt_temp = copy.deepcopy(alt)
for i in range(len(indel_st_list)):
  dif_num = startend_list[indel_st_list[i][0]][0] - indel_st_list[i][1]
  ref_num = len(ref[df_sample[df_sample['POS'] == indel_st_list[i][1]].index[0]])
  if ',' in alt_temp[df_sample[df_sample['POS'] == indel_st_list[i][1]].index[0]]:
    sep_alt = alt_temp[df_sample[df_sample['POS'] == indel_st_list[i][1]].index[0]].split(',')
    alt_num = max(map(len, sep_alt))
  else:
    alt_num = len(alt_temp[df_sample[df_sample['POS'] == indel_st_list[i][1]].index[0]])
  if dif_num <= ref_num or dif_num <= alt_num:
    indel_st_d.append(indel_st_list[i])
  
alt_temp = copy.deepcopy(alt)
for i in range(len(indel_ed_list)):
  dif_num = startend_list[indel_ed_list[i][0]][0] - indel_ed_list[i][1]
  ref_num = len(ref[df_sample[df_sample['POS'] == indel_ed_list[i][1]].index[0]])
  if ',' in alt_temp[df_sample[df_sample['POS'] == indel_ed_list[i][1]].index[0]]:
    sep_alt = alt_temp[df_sample[df_sample['POS'] == indel_ed_list[i][1]].index[0]].split(',')
    alt_num = max(map(len, sep_alt))
  else:
    alt_num = len(alt_temp[df_sample[df_sample['POS'] == indel_ed_list[i][1]].index[0]])
  if dif_num <= ref_num or dif_num <= alt_num:
    indel_ed_d.append(indel_ed_list[i])
  


start前の干渉し得るINDEL。[[n個目のエキソン,　POS番号], [n+1個目のエキソン, POS番号]...]

In [ ]:
indel_st_d

end後の干渉し得るINDEL。[[n個目のエキソン,　POS番号], [n+1個目のエキソン, POS番号]...]

In [ ]:
indel_ed_d

<h4>4.1.3 抽出した抽出したINDELが本当に干渉しているか、目視で確認する。</h4>

<p>何個目を見るか決める。１個目ならexon_numberを0にする.</p>


<p>indel_st_dの中身の[[2,********], [3, ********], [8, ********]]の場合</p>
<p>[2,********]を見るならexson_numberは0を入れる。</p>[3, ********]なら１、 [8, ********]なら２。
<p>何個目の[]内のものを見るかを選択する。</p>

<b><font color=red>$$$</font></b>

startの手前

In [ ]:
st_exon_number = 0

endの後ろ

In [ ]:
ed_exon_number = 0

<b><font color=red>$$$</font></b>

・該当するsequenceのstartのPOS番号

In [ ]:
if len(indel_st_d) > 0:
  print(startend_list[indel_st_d[st_exon_number][0]][0])

・該当するsequenceのendのPOS番号


In [ ]:
if len(indel_ed_d) > 0:
  print(startend_list[indel_ed_d[ed_exon_number][0]][1])

・POS,REF,ALTを参考に上の該当するsequenceのstartのPOS番号と比較してstart以降に干渉しているか確認。

In [ ]:
if len(indel_st_d) > 0:
  display(df_sample[df_sample['POS'] == indel_st_d[st_exon_number][1]])

・POS,REF,ALTを参考に上の該当するsequenceのendのPOS番号と比較してend以前に干渉しているか確認。

In [ ]:
if len(indel_ed_d) > 0:
  display(df_sample[df_sample['POS'] == indel_ed_d[ed_exon_number][1]])

<font color='red'>※干渉しているかのチェックのみで干渉している場合の処理は実装していません。</font>

<h3>4.2 kd_m_gt,f1_f_gtの処理</h3>

<p>・変異の有無のnanの処理。</p>

1.   '-'ならleft=0,right=0
2.   left>=0.9ならleft=0,right=0

1.   right>=0.9ならleft=1,right=1
2.   それ以外ならleft=0,right=1





閾値を0.9で設定。変更する場合thresholdの値を調整する。

<b><font color=red>$$$</font></b>

In [ ]:
threshold = 0.9

<b><font color=red>$$$</font></b>

・kd_m_gt,f1_f_gt左右でリスト作成。nanの前処理

In [ ]:
kd_m_gt.fillna('N|N', inplace=True)
f1_f_gt.fillna('N|N', inplace=True)
for i in range(len(f1_f_gt)):
    f1_f_gt[i] = str(f1_f_gt[i])
    f1_f_gt[i] = f1_f_gt[i].split('|')      
for i in range(len(kd_m_gt)):
    kd_m_gt[i] = str(kd_m_gt[i])
    kd_m_gt[i] = kd_m_gt[i].split('|')
kd_m_gt1 = []
kd_m_gt2 = []
f1_f_gt1 = []
f1_f_gt2 = []
for i in range(len(kd_m_gt)):
    kd_m_gt1.append(kd_m_gt[i][0])
for i in range(len(kd_m_gt)):
    kd_m_gt2.append(kd_m_gt[i][1])
for i in range(len(f1_f_gt)):
    f1_f_gt1.append(f1_f_gt[i][0])
for i in range(len(f1_f_gt)):
    f1_f_gt2.append(f1_f_gt[i][1])

・kd_m_gt1,kd_m_gt2,f1_f_gt1,f1_f_gt2にnan処理を適用したものを作成する関数。

In [ ]:
def kdf1_na(genotype_list,kdf1left,kdf1right , genoLeft, genoRight ,threshold):
  for i in range(len(genotype_list)):
    if genotype_list[i][0] == 'N' and genotype_list[i][1] == 'N':
      if genoLeft[i] == '-' or genoRight[i] == '-':
        kdf1left[i] = '0'
        kdf1right[i] = '0'
      else:
        left = float(genoLeft[i])
        right = float(genoRight[i])
        if left >= threshold:
          kdf1left[i] = '0'
          kdf1right[i] = '0'
        elif right >= threshold:
          kdf1left[i] = '1'
          kdf1right[i] = '1'
        else:
          kdf1left[i] = '0'
          kdf1right[i] = '1'
      
  return kdf1left, kdf1right

・上記の実行コード

In [ ]:
kd_m_gt1,kd_m_gt2= kdf1_na(genotype_list=kd_m_gt,kdf1left=kd_m_gt1,kdf1right=kd_m_gt2 , genoLeft=kd_m_af_genotype_0, genoRight=kd_m_af_genotype_1,threshold=threshold)
f1_f_gt1,f1_f_gt2= kdf1_na(genotype_list=f1_f_gt,kdf1left=f1_f_gt1,kdf1right=f1_f_gt2 , genoLeft=f1_f_af_genotype_0, genoRight=f1_f_af_genotype_1,threshold=threshold)

<h3>4.3 SNPorINDELの位置特定</h3>

<p>・SNPorINDELがどの領域に含まれているかを特定。</p>
<p>snp_list:sequenceの数分[ ]を作成。各[ ]内に該当sequence内の変異が起きている起きているPOS番号を入れる。</p>

In [ ]:
snp_list = [[] for i in range(len(startend_list))]
for i in range(len(startend_list)):
  for j in pos:
    if startend_list[i][0] <= j <= startend_list[i][1]:
      snp_list[i].append(int(j))

<font color='blue'>snp_listを表示</font>

In [ ]:
snp_list

<h3>4.4 変異の適用</h3>

<h4>4.4.1 ALTが2パターン以上ある場合の処理</h4>

・変異が0or1以外のものを特定し、altリスト内の該当箇所を,で区切る'aaa,bbb'→[aaa,bbb]のように

In [ ]:
dif = [] #dif : どれかが0又は1以外のidx番号
for i in range(len(kd_m_gt1)):
  if kd_m_gt1[i] != '0' and kd_m_gt1[i] != '1':
    if i not in dif:
      dif.append(i)
for i in range(len(kd_m_gt2)):
  if kd_m_gt2[i] != '0' and kd_m_gt2[i] != '1':
    if i not in dif:
      dif.append(i)
for i in range(len(f1_f_gt1)):
  if f1_f_gt1[i] != '0' and f1_f_gt1[i] != '1':
    if i not in dif:
      dif.append(i)
for i in range(len(f1_f_gt2)):
  if f1_f_gt2[i] != '0' and f1_f_gt2[i] != '1':
    if i not in dif:
      dif.append(i)
for i in dif:
  alt[i] = alt[i].split(',')

<h4>4.4.2 SNP,INDEL変異の適用</h4>

・SNPの変異を適用したsp_sequenceを出力する関数



In [ ]:
def refalt(num):
  temp_sp_sequence = copy.deepcopy(sp_sequence)
  for i in range(len(snp_list)):
    for j in range(len(snp_list[i])):
      idx = list(pos).index(snp_list[i][j])  
      if SNPorINDEL[idx] == 'SNP':
        if num[idx] == '0':
          a = ref[idx]
        elif idx not in dif:
            a = alt[idx]
        else:
          if num[idx] == '1':
            a = alt[idx][0]
          elif num[idx] == '2':
            a = alt[idx][1]
          elif num[idx] == '3':
            a = alt[idx][2]
        
        number = int((snp_list[i][j]-1)-(startend_list[i][0]-1))
        sp_list = [temp_sp_sequence[i][x] for x in range(len(temp_sp_sequence[i]))]
        sp_list[number] = a
        sp = ''
        for x in sp_list:
          sp += x
        temp_sp_sequence[i] = sp

  return temp_sp_sequence

<p>・INDELの変異を適用したsp_sequenceを出力する関数.</p>
<p>欠失…該当文字数を0に変換。変異後を該当文字列の一番最初にすべて入れる。0が残るため番号は変化しない</p>
<p>挿入…該当文字数を0に変換。変異後を該当文字列の一番最初にすべて入れる。こちらも番号は変化しない</p>

In [ ]:
def refalt2(snped_sequence, numlist):
  sp_snped_sequence = []
  for i in snped_sequence:
    sp_list = [i[x] for x in range(len(i))]
    sp_snped_sequence.append(sp_list)

  for i in range(len(snp_list)):
    for j in range(len(snp_list[i])):
      idx = list(pos).index(snp_list[i][j])  
      if SNPorINDEL[idx] == 'INDEL':
        if numlist[idx] == '0':
          a = ref[idx]
        elif idx not in dif:
          a = alt[idx]
        else:
          if numlist[idx] == '1':
            a = alt[idx][0]
          elif numlist[idx] == '2':
            a = alt[idx][1]
          elif numlist[idx] == '3':
            a = alt[idx][2]

        number = int((snp_list[i][j])-(startend_list[i][0]))
        count = len(ref[idx])
        outcount = number+len(ref[idx])-1-(startend_list[i][1]-startend_list[i][0])
        #欠失
        if len(ref[idx]) >= len(a):
          if outcount > 0:
            count -= outcount
            a = a[:count]
            print(ref[idx], count, a)
          for k in range(0,count):
            sp_snped_sequence[i][number+k] = '0'
          sp_snped_sequence[i][number] = a

        #挿入
        if len(ref[idx]) <= len(a):
          if outcount > 0:
            count -= outcount
            a = a[:count]
            print(ref[idx], count, a)
          for k in range(0, count):
            sp_snped_sequence[i][number+k] = '0'
          sp_snped_sequence[i][number] = a            

  return sp_snped_sequence
        

・0を消してmerge_sequenceを作成する関数

In [ ]:
def merge_sp(squence):
  all = ''
  for i in squence:
    for j in i:
      if j != '0':
        all += j
  all = all.upper()
  return all

・SNP変異適用処理

In [ ]:
k1snp = refalt(num=kd_m_gt1)
k2snp = refalt(num=kd_m_gt2)
f1snp = refalt(num=f1_f_gt1)
f2snp = refalt(num=f1_f_gt2)

・INDEL変異適用処理

SNP処理後の配列にINDEL処理適用

In [ ]:
k1indel = refalt2(snped_sequence = k1snp, numlist = kd_m_gt1)
k2indel = refalt2(snped_sequence = k2snp, numlist = kd_m_gt2)
f1indel = refalt2(snped_sequence = f1snp, numlist = f1_f_gt1)
f2indel = refalt2(snped_sequence = f2snp, numlist = f1_f_gt2)

・merge処理

In [ ]:
k1 = merge_sp(k1indel)
k2 = merge_sp(k2indel)
f1 = merge_sp(f1indel)
f2 = merge_sp(f2indel)

<h4>4.4.3 マイナスの場合の処理

<font color='red'>※プラスの場合でも問題ないので実行してください。</font>

・塩基の対応表

In [ ]:
dna_dict = {'A':'T', 'T':'A', 'C':'G', 'G':'C'} #-の時の対応表

・マイナスの時、ATCGと順番を入れ替えたmerge_sequenceを出力する関数

In [ ]:
#-の時の処理
def minus_ad(minus_seq):
  if df_group.loc[df_group.index[idx_num], '+'] == '-':
    p = ''
    for i in range(len(minus_seq)):
      amino = dna_dict[minus_seq[i]]
      p = amino + p
    return p
  else:
    return minus_seq

・マイナスの場合の適用処理

In [ ]:
k1seq = minus_ad(k1)
k2seq = minus_ad(k2)
f1seq = minus_ad(f1)
f2seq = minus_ad(f2)

<h3>4.5 変更を適用したsequenceを保存する</h3>

・merge_sequenceのテキストファイルを出力する関数

In [ ]:
def seq_txt(seq, name, dir):
  os.makedirs(f'{dir}', exist_ok=True)
  with open(f'{dir}/{name}.txt', 'w', encoding='utf-8') as f:
    f.write(seq)

<p>塩基配列のテキストファイルを出力するコード。</p>
<p>merge_dirはsyn_detect/workの中に作られるフォルダ。好きなフォルダ名に設定可能。デフォルトはmerge_sequence</p>
<p>nameはテキストファイルの名前</p>


全て同じ名前で実行すると上書きされていくのでほしいものはダウンロードするなりフォルダ名を変えるなりする。

<b><font color=red>$$$</font></b>

In [ ]:
merge_dir = 'merge_sequence'
dir = f'{work_dir}/{chromosome_number}/{target_name}/{id_list[id_list_n]}'
os.makedirs(dir, exist_ok=True)
seq_txt(seq=k1seq, name='k1seq',dir=f'{dir}/{merge_dir}')
seq_txt(seq=k2seq, name='k2seq',dir=f'{dir}/{merge_dir}')
seq_txt(seq=f1seq, name='f1seq',dir=f'{dir}/{merge_dir}')
seq_txt(seq=f2seq, name='f2seq',dir=f'{dir}/{merge_dir}')

<b><font color=red>$$$</font></b>

<h2>5. アミノ酸配列へ変換</h2>

4で作成した変更適用塩基配列のテキストファイルを読み込む。

In [ ]:
def open_txtfile(path,dir):
  with open(f'{dir}/{path}', 'r', encoding='utf-8') as f:
    code = f.read()
  return code

In [ ]:
k1seq = open_txtfile(path='k1seq.txt',dir=f'{dir}/{merge_dir}')
k2seq = open_txtfile(path='k2seq.txt',dir=f'{dir}/{merge_dir}')
f1seq = open_txtfile(path='f1seq.txt',dir=f'{dir}/{merge_dir}')
f2seq = open_txtfile(path='f2seq.txt',dir=f'{dir}/{merge_dir}')

コドンとアミノ酸の対応表</p>
終始コドンにはハイフンを挿入

In [ ]:
codon_dict = {'TTT' : 'F', 'TCT' : 'S', 'TAT' : 'Y', 'TGT' : 'C',
              'TTC' : 'F', 'TCC' : 'S', 'TAC' : 'Y', 'TGC' : 'C',
              'TTA' : 'L', 'TCA' : 'S', 'TAA' : '-', 'TGA' : '-',
              'TTG' : 'L', 'TCG' : 'S', 'TAG' : '-', 'TGG' : 'W',

              'CTT' : 'L', 'CCT' : 'P', 'CAT' : 'H', 'CGT' : 'R',
              'CTC' : 'L', 'CCC' : 'P', 'CAC' : 'H', 'CGC' : 'R',
              'CTA' : 'L', 'CCA' : 'P', 'CAA' : 'Q', 'CGA' : 'R',
              'CTG' : 'L', 'CCG' : 'P', 'CAG' : 'Q', 'CGG' : 'R',

              'ATT' : 'I', 'ACT' : 'T', 'AAT' : 'N', 'AGT' : 'S',
              'ATC' : 'I', 'ACC' : 'T', 'AAC' : 'N', 'AGC' : 'S',
              'ATA' : 'I', 'ACA' : 'T', 'AAA' : 'K', 'AGA' : 'R',
              'ATG' : 'M', 'ACG' : 'T', 'AAG' : 'K', 'AGG' : 'R',

              'GTT' : 'V', 'GCT' : 'A', 'GAT' : 'D', 'GGT' : 'G',
              'GTC' : 'V', 'GCC' : 'A', 'GAC' : 'D', 'GGC' : 'G',
              'GTA' : 'V', 'GCA' : 'A', 'GAA' : 'E', 'GGA' : 'G',
              'GTG' : 'V', 'GCG' : 'A', 'GAG' : 'E', 'GGG' : 'G'}

・merge_sequenceをコドンアミノ酸対応表を基にアミノ酸に変換する関数。対応表に無いものは?を挿入する

In [ ]:
def dna2protein(dna, codon_dict):
  p = ''
  for i in range(int(len(dna)/3)):
    codon = dna[3*i:3*i+3]
    if codon in codon_dict:
      p += codon_dict[codon]
    else:
      p += '?'
  
  if '?' in p:
    print('?')
  
  return p

アミノ酸変換実行コード</p>
?が表示されたらコドン表に無い３つの組み合わせが出てることになる

In [ ]:
k1p = dna2protein(dna=k1seq, codon_dict=codon_dict)
k2p = dna2protein(dna=k2seq, codon_dict=codon_dict)
f1p = dna2protein(dna=f1seq, codon_dict=codon_dict)
f2p = dna2protein(dna=f2seq, codon_dict=codon_dict)

<font color='blue'>各配列を表示</font>

In [ ]:
k1p

In [ ]:
k2p

In [ ]:
f1p

In [ ]:
f2p

・アミノ酸配列に変換したものをテキストファイルで出力する

テキストファイル出力の時と要領は同じ。

<b><font color=red>$$$</font></b>

In [ ]:
protein_dir = 'protein'
seq_txt(seq=k1p, name='k1p',dir=f'{dir}/{protein_dir}')
seq_txt(seq=k2p, name='k2p',dir=f'{dir}/{protein_dir}')
seq_txt(seq=f1p, name='f1p',dir=f'{dir}/{protein_dir}')
seq_txt(seq=f2p, name='f2p',dir=f'{dir}/{protein_dir}')

<b><font color=red>$$$</font></b>

<h2>7.アミノ酸の変異前後の特定、リスト化</h2>

k1=k2 and f1!=f2 and k1=f1 or k1=f2に該当するものを抽出

In [ ]:
target_snp = [[] for i in range(len(snp_list))]
for i in range(len(snp_list)):
  for j in range(len(snp_list[i])):
    idx = df_sample[df_sample['POS'] == snp_list[i][j]].index[0]
    k1pl = kd_m_gt1[idx]
    k2pl = kd_m_gt2[idx]
    f1pl = f1_f_gt1[idx]
    f2pl = f1_f_gt2[idx]
    if k1pl == k2pl and f1pl != f2pl:
      if k1pl == f1pl or k1pl == f2pl:
        target_snp[i].append([snp_list[i][j],[k1pl,k2pl], [f1pl, f2pl]]) 

<font color='blue'>K1=K2=F1又はK1=K2=F2のものを表示</font>

ない場合はどこかでエラーが出ます。
途中まではデータが出力されます。

[POS番号, K1, K2, F1, F2]

In [ ]:
target_snp

・kとfの変更前後の塩基、コドン、アミノ酸を特定する関数

In [ ]:
def first_detect(target_snp, startend_list, seq_indel, all_seq, protein):
  #K側の特定
  target = copy.deepcopy(target_snp)
  count = 0
  diff_sum = 0
  indel_length = 0
  for i in range(len(target)):
    count_temp = count
    for j in range(len(target[i])):
      idx = df_sample[df_sample['POS'] == target[i][j][0]].index[0]
      if SNPorINDEL[idx] == 'SNP':
        number = (target[i][j][0])-(startend_list[i][0])
        if df_group.loc[df_group.index[0], '+'] == '-':
          target[i][j].append([dna_dict[seq_indel[i][number]]])
        else:
          target[i][j].append([seq_indel[i][number]])
        codon_num = count_temp+diff_sum+number
        one_codon,one_p = codon_number(codon_num=codon_num, all_seq = all_seq, indel_length=indel_length, idx=idx)
        target[i][j].append([one_codon])
        target[i][j].append([one_p])

      else: #INDEL
        number = (target[i][j][0])-(startend_list[i][0])
        if df_group.loc[df_group.index[0], '+'] == '-':
          indel_base = seq_indel[i][number]
          indel_base = ch_amino(indel_base.upper())
        else:
          indel_base = seq_indel[i][number]
        target[i][j].append([''.join(list(reversed(indel_base)))])
        target[i][j].append([])
        target[i][j].append([])
        if len(indel_base)%3 == 0:
          indel_num = len(indel_base)//3 + 1
        else:
          indel_num = len(indel_base)//3 + 2
        indel_codon_num = count_temp+diff_sum+number
        indel_length = len(target[i][j][3][0])
        for num in range(indel_num):
          one_codon,one_p = codon_number(codon_num=indel_codon_num, all_seq = all_seq, indel_length=indel_length, idx=idx)
          target[i][j][4].append(one_codon)
          target[i][j][5].append(one_p)
          if df_group.loc[df_group.index[0], '+'] == '-':
            indel_codon_num -= 3
          else:
            indel_codon_num += 3
        
        original_ref = ref[df_sample[df_sample['POS'] == target[i][j][0]].index[0]]
        diff = len(original_ref) - len(indel_base)
        if len(original_ref) > len(indel_base):
          count_temp -= diff
          diff_sum -= diff
        elif len(original_ref) < len(indel_base):
          count_temp -= diff
          diff_sum -= diff
        else:
          pass

    count += len(sp_sequence[i])
  
  return target


def second_detect(target_snp, startend_list, seq_indel, all_seq, protein):
  #F側の特定
  target = copy.deepcopy(target_snp)
  count = 0
  diff_sum = 0
  indel_length = 0
  for i in range(len(target)):
    count_temp = count
    for j in range(len(target[i])):
      idx = df_sample[df_sample['POS'] == target[i][j][0]].index[0]
      if SNPorINDEL[idx] == 'SNP':
        number = (target[i][j][0])-(startend_list[i][0])
        if df_group.loc[df_group.index[0], '+'] == '-':
          target[i][j][3].append(dna_dict[seq_indel[i][number]])
        else:
          target[i][j][3].append(seq_indel[i][number])
        codon_num = count_temp+number+diff_sum
        one_codon,one_p = codon_number(codon_num=codon_num, all_seq = all_seq, indel_length=indel_length, idx=idx)
        target[i][j][4].append(one_codon)
        target[i][j][5].append(one_p)

      else:
        number = (target[i][j][0])-(startend_list[i][0])
        if df_group.loc[df_group.index[0], '+'] == '-':
          indel_base = seq_indel[i][number]
          indel_base = ch_amino(indel_base.upper())
        else:
          indel_base = seq_indel[i][number]
        target[i][j][3].append(''.join(list(reversed(indel_base))))
        target[i][j][4].append('|')
        target[i][j][5].append('|')
        if len(indel_base)%3 == 0:
          indel_num = len(indel_base)//3 + 1
        else:
          indel_num = len(indel_base)//3 + 2
        indel_codon_num = count_temp+diff_sum+number
        indel_length = len(target[i][j][3][1])        
        for num in range(indel_num):
          one_codon,one_p = codon_number(codon_num=indel_codon_num, all_seq = all_seq, indel_length=indel_length, idx=idx)
          target[i][j][4].append(one_codon)
          target[i][j][5].append(one_p)
          if df_group.loc[df_group.index[0], '+'] == '-':
            indel_codon_num -= 3
          else:
            indel_codon_num += 3
        original_ref = ref[df_sample[df_sample['POS'] == target[i][j][0]].index[0]]
        diff = len(original_ref) - len(indel_base)
        if len(original_ref) > len(indel_base):
          count_temp -= diff
          diff_sum -= diff
        elif len(original_ref) < len(indel_base):
          count_temp -= diff
          diff_sum -= diff
        else:
          pass 
      
    count += len(sp_sequence[i])
  
  return target

・コドンの変換に関わる関数

In [ ]:
def codon_number(codon_num,all_seq,indel_length, idx):
  if df_group.loc[df_group.index[0], '+'] == '+':
    if (codon_num)%3 == 0:
      one_codon = all_seq[codon_num:codon_num+3]
    elif (codon_num)%3 == 1:
      one_codon = all_seq[codon_num-1:codon_num+2]
    elif (codon_num)%3 == 2:
      one_codon = all_seq[codon_num-2:codon_num+1]
  
  elif df_group.loc[df_group.index[0], '+'] == '-' and SNPorINDEL[idx] == 'SNP':
    if (codon_num)%3 == 0:
      one_codon = dna_dict[all_seq[codon_num+2]] + dna_dict[all_seq[codon_num+1]] + dna_dict[all_seq[codon_num]]
    elif (codon_num)%3 == 1:
      one_codon = dna_dict[all_seq[codon_num+1]] + dna_dict[all_seq[codon_num]] + dna_dict[all_seq[codon_num-1]]
    elif (codon_num)%3 == 2:
      one_codon = dna_dict[all_seq[codon_num]] + dna_dict[all_seq[codon_num-1]] + dna_dict[all_seq[codon_num-2]]
      
  elif df_group.loc[df_group.index[0], '+'] == '-' and SNPorINDEL[idx] == 'INDEL':
    m_seq = ch_amino(minus_seq=all_seq)
    m_seq = ''.join(list(reversed(m_seq)))
    rev_num = len(m_seq)-codon_num-indel_length
    if (rev_num)%3 == 0:
      one_codon = m_seq[rev_num:rev_num+3]
    elif (rev_num)%3 == 1:
      one_codon = m_seq[rev_num-1:rev_num+2]
    elif (rev_num)%3 == 2:
      one_codon = m_seq[rev_num-2:rev_num+1]
  
  one_p = codon_dict[one_codon]

  return one_codon, one_p

def ch_amino(minus_seq):
  p = ''
  for i in range(len(minus_seq)):
    amino = dna_dict[minus_seq[i]]
    p = p + amino
  return p

変更前の特定。k1を用いて行なっている。

In [ ]:
k1_detect = first_detect(target_snp=target_snp, startend_list=startend_list, seq_indel=k1indel, all_seq=k1, protein=k1p)

[POS番号, [k1, k2], [f1, f2], [塩基], [コドン], [アミノ酸]]

In [ ]:
k1_detect

・k1と異なる、変更後の各データを出力する。

k1とf1が異なる場合又は、k1とf2が異なる場合を検知し、変更後のデータを加える。

In [ ]:
for i in k1_detect:
  if len(i) > 0:
    d = i[0]
    break

In [ ]:
if d[1][0] != d[2][0]:
  detected = second_detect(target_snp=k1_detect, startend_list=startend_list, seq_indel=f1indel, all_seq=f1, protein=f1p)
elif  d[1][0] != d[2][1]:
  detected = second_detect(target_snp=k1_detect, startend_list=startend_list, seq_indel=f2indel, all_seq=f2, protein=f2p)

<font color='blue'>変更前後を表示</font>

[POS番号, [k1, k2], [f1, f2], [塩基, 変更後の塩基], [コドン, 変更後のコドン], [アミノ酸, 変更後のアミノ酸]

In [ ]:
detected

特定した変異をエクセルファイルに出力する関数

In [ ]:
def writeexcel(detect_list_row, xlsxname, dir):
  detect_list = []
  for i in range(len(detect_list_row)):
    if len(detect_list_row[i]) == 0:
      pass
    else:
      detect_list += (detect_list_row[i])

  for i in range(len(detect_list)):
    for j in range(len(detect_list[i])):
      if type(detect_list[i][j]) == list and len(detect_list[i][j])==2:
        detect_list[i][j] = lis2str(detect_list[i][j])
      elif type(detect_list[i][j]) == list and len(detect_list[i][j])>=3:
        detect_list[i][j] = indellis2str(detect_list[i][j])
  wb=openpyxl.Workbook()
  ws = wb['Sheet']
  for i in range(len(detect_list)):
    for j in range(len(detect_list[i])):
      ws.cell(i+3,j+1,value = detect_list[i][j])
  
  ws.cell(1,1,value = id_list[0])
  ws.cell(2,1,value = 'POS')
  ws.cell(2,2,value = 'KD_M_GT')
  ws.cell(2,3,value = 'F1_F_GT')
  ws.cell(2,4,value = 'kd/f1')
  ws.cell(2,5,value = 'codon')
  ws.cell(2,6,value = 'protein')


  wb.save(f'{dir}/{xlsxname}.xlsx')

def lis2str(lis2):
  st = str(lis2[0]+'/'+lis2[1])
  return st
  
def indellis2str(lisindel):
  st = ''
  for i in range(len(lisindel)):
    if lisindel[i] == '|':
      st = st.rstrip('/')
      st += lisindel[i]
    else:
      st += lisindel[i]
      st += '/'
  st = st.rstrip('/')
  return st

excelファイル出力

excelファイルの名前を設定する。
同じ名前のまま続けると上書きされる。

<b><font color=red>$$$</font></b>

In [ ]:
writeexcel(detect_list_row=detected, xlsxname='test', dir=dir)

<b><font color=red>$$$</font></b>